In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
my_data = pd.read_csv("/content/drive/MyDrive/archive (1)/train.csv")

In [ ]:
my_data.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [ ]:
# Combine all textual data into a single column
my_data['total_text'] = my_data['TITLE'] + my_data['ABSTRACT']

In [ ]:
my_data.set_index('ID', inplace = True)
my_data.head()

,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,total_text
ID,,,,,,,,,
1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps P...
2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0,Rotation Invariance Neural Network Rotation i...
3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...
4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0,A finite element approximation for the stochas...
5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0,Comparative study of Discrete Wavelet Transfor...


In [ ]:
my_data = my_data[['total_text', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']]

In [ ]:
my_data.head()

,total_text,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
ID,,,,,,,
1,Reconstructing Subject-Specific Effect Maps P...,1,0,0,0,0,0
2,Rotation Invariance Neural Network Rotation i...,1,0,0,0,0,0
3,Spherical polyharmonics and Poisson kernels fo...,0,0,1,0,0,0
4,A finite element approximation for the stochas...,0,0,1,0,0,0
5,Comparative study of Discrete Wavelet Transfor...,1,0,0,1,0,0


In [ ]:
# Clean the textual column
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import re
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: word_tokenize(x))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: word_lemmatizer(x))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join(x))
    return df

In [ ]:
total_rows = len(my_data)


train_size = int(total_rows * 0.8)
valid_size = int(total_rows * 0.1)

df_train = my_data[:train_size]
df_valid = my_data[train_size : train_size+valid_size]
test_df = my_data[train_size+valid_size:]

In [ ]:
train_clean_df = clean_text(df_train, 'total_text', 'total_text_clean')

<ipython-input-8-863e1a1ab39d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_text_field_name] = df[text_field].str.lower()
<ipython-input-8-863e1a1ab39d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
<ipython-input-8-863e1a1ab39d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [ ]:
final_df = train_clean_df[['total_text_clean', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']]
final_df.head(5)

,total_text_clean,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
ID,,,,,,,
1,reconstructing subjectspecific effect map pred...,1,0,0,0,0,0
2,rotation invariance neural network rotation in...,1,0,0,0,0,0
3,spherical polyharmonics poisson kernel polyhar...,0,0,1,0,0,0
4,finite element approximation stochastic maxwel...,0,0,1,0,0,0
5,comparative study discrete wavelet transforms ...,1,0,0,1,0,0


In [ ]:
final_df['labels'] = final_df.apply(lambda x: list([x['Computer Science'], x['Physics'], x['Mathematics'], x['Statistics'], x['Quantitative Biology'], x['Quantitative Finance']]),axis=1)
final_df.head()

,total_text_clean,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,labels
ID,,,,,,,,
1,reconstructing subjectspecific effect map pred...,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]"
2,rotation invariance neural network rotation in...,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]"
3,spherical polyharmonics poisson kernel polyhar...,0,0,1,0,0,0,"[0, 0, 1, 0, 0, 0]"
4,finite element approximation stochastic maxwel...,0,0,1,0,0,0,"[0, 0, 1, 0, 0, 0]"
5,comparative study discrete wavelet transforms ...,1,0,0,1,0,0,"[1, 0, 0, 1, 0, 0]"


In [ ]:
train_df = final_df[['total_text_clean','labels']]
train_df.columns = ['text','labels']
train_df.head()

,text,labels
ID,,
1,reconstructing subjectspecific effect map pred...,"[1, 0, 0, 0, 0, 0]"
2,rotation invariance neural network rotation in...,"[1, 0, 0, 0, 0, 0]"
3,spherical polyharmonics poisson kernel polyhar...,"[0, 0, 1, 0, 0, 0]"
4,finite element approximation stochastic maxwel...,"[0, 0, 1, 0, 0, 0]"
5,comparative study discrete wavelet transforms ...,"[1, 0, 0, 1, 0, 0]"


In [ ]:
!pip install simpletransformers

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel

In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 20.1 MB/s eta 0:00:00


In [ ]:
# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    "xlm",
    "xlm-mlm-en-2048",
    num_labels=6,
    args={"reprocess_input_data": True, "overwrite_output_dir": True, "num_train_epochs": 5},
)

Some weights of XLMForMultiLabelSequenceClassification were not initialized from the model checkpoint at xlm-mlm-en-2048 and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
model.train_model(train_df)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 5:   0%|          | 0/2098 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Running Epoch 2 of 5:   0%|          | 0/2098 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/2098 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/2098 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
test_df

,total_text,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
ID,,,,,,,
18875,The Structural Fate of Individual Multicompone...,0,1,0,0,0,0
18876,On Compression of Unsupervised Neural Nets by ...,1,0,0,0,0,0
18877,A matrix generalization of a theorem of Fine ...,1,0,1,0,0,0
18878,Training Multi-Task Adversarial Network For Ex...,1,0,0,0,0,0
18879,Playing Games with Bounded Entropy In this pa...,1,0,0,0,0,0
...,...,...,...,...,...,...,...
20968,Contemporary machine learning: a guide for pra...,1,1,0,0,0,0
20969,Uniform diamond coatings on WC-Co hard alloy c...,0,1,0,0,0,0
20970,Analysing Soccer Games with Clustering and Con...,1,0,0,0,0,0


In [ ]:
test_clean_df = clean_text(test_df, 'total_text', 'total_text_clean')

<ipython-input-8-863e1a1ab39d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_text_field_name] = df[text_field].str.lower()
<ipython-input-8-863e1a1ab39d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
<ipython-input-8-863e1a1ab39d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [ ]:
test_clean_df = test_clean_df[['total_text_clean']]
test_clean_df.head()

,total_text_clean
ID,
18875,structural fate individual multicomponent meta...
18876,compression unsupervised neural net pruning we...
18877,matrix generalization theorem fine nathan fine...
18878,training multitask adversarial network extract...
18879,playing game bounded entropy paper consider ze...


In [ ]:
test_clean_df.shape

(2098, 1)

In [ ]:
prediction_list = test_clean_df.total_text_clean.tolist()

In [ ]:
predictions, raw_outputs = model.predict(prediction_list)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:2188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [ ]:
test_clean_df['predictions'] = predictions

<ipython-input-26-c5e270ec5fdd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_clean_df['predictions'] = predictions


In [ ]:
test_clean_df[['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']] = pd.DataFrame(test_clean_df['predictions'].tolist(), index= test_clean_df.index)

<ipython-input-27-1816804b298b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_clean_df[['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']] = pd.DataFrame(test_clean_df['predictions'].tolist(), index= test_clean_df.index)
<ipython-input-27-1816804b298b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_clean_df[['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']] = pd.DataFrame(test_

In [ ]:
test_clean_df.head()

In [ ]:
submission_df = test_clean_df[['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']]

In [ ]:
submission_df.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
ID,,,,,,
18875,0,1,0,0,0,0
18876,1,0,0,1,0,0
18877,0,0,1,0,0,0
18878,1,0,0,1,0,0
18879,1,0,0,0,0,0


In [ ]:
submission_df.to_csv("submission_df.csv")

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
column_names=['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']

pred=test_clean_df
acc_scores = []
f1_scores = []
for i, col_name in enumerate(column_names):
    acc = accuracy_score(test_df[col_name], pred[col_name]).item()
    f1 = f1_score(test_df[col_name], pred[col_name], average='macro')

    acc_scores.append(acc)
    f1_scores.append(f1)

    print (col_name)
    print ('acc: {:.3f}'.format(acc))
    print ('f1 : {:.3f}'.format(f1))
    print ()

Computer Science
acc: 0.581
f1 : 0.368

Physics
acc: 0.715
f1 : 0.417

Mathematics
acc: 0.735
f1 : 0.423

Statistics
acc: 0.751
f1 : 0.429

Quantitative Biology
acc: 0.978
f1 : 0.494

Quantitative Finance
acc: 0.986
f1 : 0.497



In [ ]:
accuracy=sum(acc_scores)/len(acc_scores)
accuracy

0.7909119796631713